In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torchsummary import summary


class GraphNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(GraphNeuralNetwork, self).__init__()
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
        self.conv3 = GCNConv(hidden_size, hidden_size)
        self.conv4 = GCNConv(hidden_size, hidden_size)
        self.conv5 = GCNConv(hidden_size, output_size)
        self.num_layers = num_layers

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # GNN layers
        for i in range(self.num_layers):
            x = F.relu(getattr(self, f'conv{i + 1}')(x, edge_index))

        return F.log_softmax(x, dim=1)

# Assuming you have a dataset with input and output points
# You should convert your data to PyTorch Geometric format, e.g., using `torch_geometric.data.Data` class

# Example data preparation
from torch_geometric.data import Data

# Assuming your input and output points are of size 672
input_size = 672

num_points = 672
input_points = torch.randn(num_points, input_size)  # Random input data
output_points = torch.randint(0, 2, (num_points,))   # Random binary classification labels

# Construct PyTorch Geometric Data object
edge_index = torch.zeros((2, num_points), dtype=torch.long)
for i in range(num_points):
    edge_index[0, i] = i
    edge_index[1, i] = (i + 1) % num_points

data = Data(x=input_points, edge_index=edge_index, y=output_points)

# Instantiate the GNN model
hidden_size = 128
output_size = 2  # Binary classification
num_layers = 5

model = GraphNeuralNetwork(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# # Training loop
# num_epochs = 100
# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     output = model(data)
#     loss = criterion(output, data.y)
#     loss.backward()
#     optimizer.step()
# 
#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# After training, you can use the trained model for predictions on new data
# For example, you can do something like:
# new_data = ...  # Prepare new data in PyTorch Geometric format
# predicted_labels = model(new_data).argmax(dim=1)


In [3]:
summary(model, input_size=(input_size,), device='cuda' if torch.cuda.is_available() else 'cpu')

AttributeError: 'Tensor' object has no attribute 'x'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GraphNet(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(GraphNet, self).__init__()

        # Define graph convolutional layers
        self.conv1 = GCNConv(input_size, hidden_sizes[0])
        self.conv2 = GCNConv(hidden_sizes[0], hidden_sizes[1])
        self.conv3 = GCNConv(hidden_sizes[1], hidden_sizes[2])
        self.conv4 = GCNConv(hidden_sizes[2], hidden_sizes[3])
        self.conv5 = GCNConv(hidden_sizes[3], hidden_sizes[4])

        # Fully connected layer for regression
        self.fc = nn.Linear(hidden_sizes[4], output_size)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply graph convolutional layers
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))

        # Global mean pooling to get a fixed-size output
        x = torch.mean(x, dim=0)

        # Fully connected layer for regression
        x = self.fc(x)

        return x

# Define your dataset and DataLoader here (assuming you have a PyTorch Geometric dataset)

# Example usage:
input_size = 672
hidden_sizes = [128, 128, 64, 32, 16]
output_size = 6  # Assuming you want to regress 6 parameters
model = GraphNet(input_size, hidden_sizes, output_size)

# Define your loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train your model using your dataset and DataLoader
# for epoch in range(num_epochs):
#     for batch_data in dataloader:
#         optimizer.zero_grad()
#         output = model(batch_data)
#         loss = criterion(output, batch_data.y)
#         loss.backward()
#         optimizer.step()


In [19]:
from torch_geometric.nn import summary

summary(model, data, max_depth=10)


'+--------------------+----------------------+----------------+----------+\n| Layer              | Input Shape          | Output Shape   | #Param   |\n|--------------------+----------------------+----------------+----------|\n| GraphNeuralNetwork | [672, 672]           | [672, 2]       | 135,938  |\n| ├─(conv1)GCNConv   | [672, 672], [2, 672] | [672, 128]     | 86,144   |\n| ├─(conv2)GCNConv   | [672, 128], [2, 672] | [672, 128]     | 16,512   |\n| ├─(conv3)GCNConv   | [672, 128], [2, 672] | [672, 128]     | 16,512   |\n| ├─(conv4)GCNConv   | [672, 128], [2, 672] | [672, 128]     | 16,512   |\n| ├─(conv5)GCNConv   | [672, 128], [2, 672] | [672, 2]       | 258      |\n+--------------------+----------------------+----------------+----------+'

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GraphNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(GraphNeuralNetwork, self).__init__()
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
        self.conv3 = GCNConv(hidden_size, hidden_size)
        self.conv4 = GCNConv(hidden_size, hidden_size)
        self.conv5 = GCNConv(hidden_size, output_size)
        self.num_layers = num_layers

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # GNN layers
        for i in range(self.num_layers):
            x = F.relu(getattr(self, f'conv{i + 1}')(x, edge_index))

        return F.log_softmax(x, dim=1)

# Assuming you have a dataset with input and output points
# You should convert your data to PyTorch Geometric format, e.g., using `torch_geometric.data.Data` class

# Example data preparation
from torch_geometric.data import Data

# Assuming your input and output points are of size 672
num_points = 672
input_points = torch.randn(num_points, input_size)  # Random input data
output_points = torch.randint(0, 2, (num_points,))   # Random binary classification labels

# Construct PyTorch Geometric Data object
edge_index = torch.zeros((2, num_points), dtype=torch.long)
for i in range(num_points):
    edge_index[0, i] = i
    edge_index[1, i] = (i + 1) % num_points

data = Data(x=input_points, edge_index=edge_index, y=output_points)

# Instantiate the GNN model
input_size = 672
hidden_size = 128
output_size = 2  # Binary classification
num_layers = 5

model = GraphNeuralNetwork(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
# num_epochs = 100
# for epoch in range(num_epochs):
#     optimizer.zero_grad()
#     output = model(data)
#     loss = criterion(output, data.y)
#     loss.backward()
#     optimizer.step()
# 
#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GNNRegressionModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(GNNRegressionModel, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size

        # Define graph convolutional layers
        self.conv1 = GCNConv(input_size, hidden_sizes[0])
        self.conv2 = GCNConv(hidden_sizes[0], hidden_sizes[1])
        self.conv3 = GCNConv(hidden_sizes[1], hidden_sizes[2])
        self.conv4 = GCNConv(hidden_sizes[2], hidden_sizes[3])
        self.conv5 = GCNConv(hidden_sizes[3], hidden_sizes[4])

        # Fully connected layer for regression output
        self.fc = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Apply graph convolutional layers with activation functions
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))

        # Global mean pooling to aggregate node-level information
        x = global_mean_pool(x, batch)

        # Fully connected layer for regression output
        x = self.fc(x)

        return x

# Example usage
input_size = 672
hidden_sizes = [128, 256, 128, 64, 32]  # Adjust as needed
output_size = 6  # Number of regression parameters

# Create an instance of the GNN model
model = GNNRegressionModel(input_size, hidden_sizes, output_size)

# Print model summary
print(model)

GNNRegressionModel(
  (conv1): GCNConv(672, 128)
  (conv2): GCNConv(128, 256)
  (conv3): GCNConv(256, 128)
  (conv4): GCNConv(128, 64)
  (conv5): GCNConv(64, 32)
  (fc): Linear(in_features=32, out_features=6, bias=True)
)


In [16]:
import torchsummary


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Print model summary
summary_text = torchsummary.summary(model, (input_size,))
print(summary_text)


AttributeError: 'Tensor' object has no attribute 'x'